In [1]:
import lance
import duckdb
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset
import shutil

## Creating datasets

Via pyarrow it's really easy to create lance datasets

Create a dataframe

In [2]:
df = pd.DataFrame({"a": [5]})
df

,a
0,5


Write it to lance

In [3]:
shutil.rmtree("/tmp/test_df.lance", ignore_errors=True)

dataset = lance.write_dataset(df, "/tmp/test_df.lance")
dataset.to_table().to_pandas()

,a
0,5


### Converting from parquet

In [4]:
shutil.rmtree("/tmp/test.parquet", ignore_errors=True)
shutil.rmtree("/tmp/test_parquet.lance", ignore_errors=True)

tbl = pa.Table.from_pandas(df)
pa.dataset.write_dataset(tbl, "/tmp/test.parquet", format='parquet')

parquet = pa.dataset.dataset("/tmp/test.parquet")
parquet.to_table().to_pandas()

,a
0,5


Write to lance in 1 line

In [5]:
dataset = lance.write_dataset(parquet, "/tmp/test_parquet.lance")

In [6]:
# make sure it's the same
dataset.to_table().to_pandas()

,a
0,5


## Versioning

We can append rows

In [7]:
df = pd.DataFrame({"a": [10]})
tbl = pa.Table.from_pandas(df)
dataset = lance.write_dataset(tbl, "/tmp/test.lance", mode="append")

dataset.to_table().to_pandas()

,a
0,50
1,100
2,10


We can overwrite the data and create a new version

In [8]:
df = pd.DataFrame({"a": [50, 100]})
tbl = pa.Table.from_pandas(df)
dataset = lance.write_dataset(tbl, "/tmp/test.lance", mode="overwrite")

In [9]:
dataset.to_table().to_pandas()

,a
0,50
1,100


The old version is still there

In [10]:
dataset.versions()

[{'version': 8,
  'timestamp': datetime.datetime(2023, 2, 13, 14, 54, 18),
  'metadata': {}},
 {'version': 9,
  'timestamp': datetime.datetime(2023, 2, 13, 14, 54, 18),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2023, 2, 13, 14, 54, 18),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2023, 2, 13, 14, 54, 18),
  'metadata': {}},
 {'version': 5,
  'timestamp': datetime.datetime(2023, 2, 13, 14, 54, 18),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2023, 2, 13, 14, 54, 18),
  'metadata': {}},
 {'version': 1,
  'timestamp': datetime.datetime(2023, 2, 13, 14, 54, 18),
  'metadata': {}},
 {'version': 6,
  'timestamp': datetime.datetime(2023, 2, 13, 14, 54, 18),
  'metadata': {}},
 {'version': 7,
  'timestamp': datetime.datetime(2023, 2, 13, 14, 54, 18),
  'metadata': {}}]

In [11]:
lance.dataset('/tmp/test.lance', version=1).to_table().to_pandas()

,a
0,5


In [12]:
lance.dataset('/tmp/test.lance', version=2).to_table().to_pandas()

,a
0,5
1,10


## Vectors

### Data preparation

For this tutorial let's use the Sift 1M dataset:

- Download `ANN_SIFT1M` from: http://corpus-texmex.irisa.fr/
- Direct link should be `ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz`
- Download and then unzip the tarball

In [13]:
!rm -rf sift* vec_data.lance
!wget ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
!tar -xzf sift.tar.gz

--2023-02-13 14:54:18--  ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
           => ‘sift.tar.gz’
Resolving ftp.irisa.fr (ftp.irisa.fr)... 131.254.254.45
Connecting to ftp.irisa.fr (ftp.irisa.fr)|131.254.254.45|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /local/texmex/corpus ... done.
==> SIZE sift.tar.gz ... 168280445
==> PASV ... done.    ==> RETR sift.tar.gz ... done.
Length: 168280445 (160M) (unauthoritative)

sift.tar.gz         100%[===================>] 160.48M  6.16MB/s    in 31s     

2023-02-13 14:54:52 (5.21 MB/s) - ‘sift.tar.gz’ saved [168280445]



Convert it to Lance

In [14]:
import struct

uri = "vec_data.lance"

with open("sift/sift_base.fvecs", mode="rb") as fobj:
    buf = fobj.read()
    data = np.array(struct.unpack("<128000000f", buf[4 : 4 + 4 * 1000000 * 128]))

    table = pa.Table.from_arrays([
        pa.array(range(1000000), type=pa.uint32()),
        pa.FixedSizeListArray.from_arrays(pa.array(data, type=pa.float32()), list_size=128)
    ], names=["id", "vector"])

    lance.write_dataset(table, uri, max_rows_per_group=8192, max_rows_per_file=1024*1024)

In [15]:
uri = "vec_data.lance"
sift1m = lance.dataset(uri)

### KNN (no index)

Sample 100 vectors as query vectors

In [16]:
import duckdb
# if this segfaults make sure duckdb v0.7+ is installed
samples = duckdb.query("SELECT vector FROM sift1m USING SAMPLE 100").to_df().vector
samples

0     [53.0, 51.0, 4.0, 0.0, 0.0, 0.0, 25.0, 52.0, 3...
1     [130.0, 13.0, 0.0, 0.0, 0.0, 9.0, 15.0, 58.0, ...
2     [8.0, 1.793662034335766e-43, 56.0, 10.0, 15.0,...
3     [47.0, 41.0, 11.0, 1.0, 1.0, 2.0, 25.0, 4.0, 1...
4     [28.0, 113.0, 89.0, 78.0, 23.0, 9.0, 39.0, 12....
                            ...                        
95    [7.0, 35.0, 102.0, 113.0, 11.0, 0.0, 0.0, 0.0,...
96    [0.0, 0.0, 0.0, 130.0, 147.0, 11.0, 0.0, 0.0, ...
97    [112.0, 126.0, 70.0, 3.0, 1.0, 0.0, 0.0, 33.0,...
98    [8.0, 16.0, 123.0, 42.0, 0.0, 0.0, 2.0, 5.0, 2...
99    [4.0, 1.793662034335766e-43, 19.0, 7.0, 60.0, ...
Name: vector, Length: 100, dtype: object

Call nearest neighbors (no ANN index here)

In [17]:
import time

start = time.time()
tbl = sift1m.to_table(columns=["id"], nearest={"column": "vector", "q": samples[0], "k": 10})
end = time.time()

print(f"Time(sec): {end-start}")
print(tbl.to_pandas())

Time(sec): 0.1079111099243164
       id                                             vector    score
0  665287  [53.0, 51.0, 4.0, 0.0, 0.0, 0.0, 25.0, 52.0, 3...      0.0
1  777904  [30.0, 24.0, 10.0, 4.0, 4.0, 16.0, 15.0, 10.0,...  79038.0
2  998236  [44.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 25.0...  83997.0
3   70597  [41.0, 33.0, 13.0, 1.0, 0.0, 0.0, 0.0, 2.0, 14...  86389.0
4  532417  [0.0, 29.0, 10.0, 30.0, 12.0, 8.0, 4.0, 21.0, ...  86972.0
5  957472  [14.0, 59.0, 24.0, 0.0, 1.0, 0.0, 0.0, 12.0, 1...  87866.0
6  274933  [10.0, 6.0, 0.0, 3.0, 12.0, 28.0, 16.0, 4.0, 1...  92838.0
7  744622  [50.0, 21.0, 8.0, 0.0, 0.0, 17.0, 15.0, 1.0, 4...  94371.0
8  815572  [0.0, 2.0, 0.0, 2.0, 6.0, 11.0, 35.0, 17.0, 6....  96181.0
9  845758  [24.0, 0.0, 0.0, 15.0, 4.0, 2.0, 15.0, 102.0, ...  97002.0


Without the index this is scanning through the whole dataset to compute the distance. <br/>

For real-time serving we can do much better with an ANN index

### Build index

Now let's build an index. We haven't implemented HNSW but IVF+PQ is shown here

**NOTE** If you'd rather not wait for index build, you can download a version with the index pre-built from [here](https://eto-public.s3.us-west-2.amazonaws.com/datasets/sift/sift_ivf256_pq16.tar.gz) and skip the next cell

In [18]:
%%time 

sift1m.create_index("vector",
                    index_type="IVF_PQ", 
                    num_partitions=256,  # IVF
                    num_sub_vectors=16)  # PQ

CPU times: user 2min 21s, sys: 3.44 s, total: 2min 24sBuilding vector index: IVF256,PQ16

Wall time: 23.4 s
Sample 65536 out of 1000000 to train kmeans of 128 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of 1000000 to train kmeans of 8 dim, 256 clusters
Sample 65536 out of

**NOTE** If you're trying this on your own data, make sure your vector (dimensions / num_sub_vectors) % 8 == 0, or else index creation will take much longer than expected due to SIMD misalignment

### Try nearest neighbors again with ANN index

Let's look for nearest neighbors again

In [19]:
sift1m = lance.dataset(uri)

In [20]:
import time

tot = 0
for q in samples:
    start = time.time()
    tbl = sift1m.to_table(nearest={"column": "vector", "q": q, "k": 10})
    end = time.time()
    tot += (end - start)

print(f"Avg(sec): {tot / len(samples)}")
print(tbl.to_pandas())

Avg(sec): 0.0009833168983459472
       id                                             vector          score
0  150287  [4.0, 1.8e-43, 19.0, 7.0, 60.0, 59.0, 3.0, 0.0...   32261.166016
1  215147  [9.0, 1.0, 3.0, 3.0, 0.0, 0.0, 55.0, 47.0, 0.0...  134006.796875
2  745853  [0.0, 0.0, 0.0, 1.0, 12.0, 17.0, 32.0, 13.0, 4...  134962.703125
3  653489  [4.0, 13.0, 8.0, 0.0, 35.0, 46.0, 21.0, 0.0, 2...  141667.125000
4   15447  [5.0, 0.0, 0.0, 6.0, 5.0, 27.0, 73.0, 7.0, 2.0...  142773.187500
5  130910  [1.0, 0.0, 2.0, 2.0, 12.0, 25.0, 59.0, 7.0, 1....  144413.062500
6  404906  [2.0, 0.0, 1.0, 1.0, 0.0, 0.0, 50.0, 0.0, 5.0,...  146264.437500
7  618526  [24.0, 0.0, 2.0, 40.0, 47.0, 121.0, 45.0, 4.0,...  146655.453125
8  306866  [34.0, 1.0, 0.0, 4.0, 3.0, 1.0, 35.0, 5.0, 3.0...  147506.906250
9  609371  [29.0, 21.0, 1.0, 0.0, 0.0, 0.0, 0.0, 7.0, 9.0...  148649.437500


**NOTE** on performance, your actual numbers will vary by your storage. These numbers are run on local disk on an M2 Macbook Air. If you're querying S3 directly, HDD, or network drives, performance will be slower.

The latency vs recall is tunable via:
- nprobes: how many IVF partitions to search
- refine_factor: determines how many vectors are retrieved during re-ranking

In [21]:
%%time

sift1m.to_table(nearest={"column": "vector", 
                         "q": samples[0], 
                         "k": 10, 
                         "nprobes": 10, 
                         "refine_factor": 5}).to_pandas()

CPU times: user 2.35 ms, sys: 4.22 ms, total: 6.57 ms
Wall time: 5.06 ms


,id,vector,score
0,665287,"[53.0, 51.0, 4.0, 0.0, 0.0, 0.0, 25.0, 52.0, 3...",0.0
1,777904,"[30.0, 24.0, 10.0, 4.0, 4.0, 16.0, 15.0, 10.0,...",79038.0
2,998236,"[44.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 25.0...",83997.0
3,70597,"[41.0, 33.0, 13.0, 1.0, 0.0, 0.0, 0.0, 2.0, 14...",86389.0
4,532417,"[0.0, 29.0, 10.0, 30.0, 12.0, 8.0, 4.0, 21.0, ...",86972.0
5,957472,"[14.0, 59.0, 24.0, 0.0, 1.0, 0.0, 0.0, 12.0, 1...",87866.0
6,274933,"[10.0, 6.0, 0.0, 3.0, 12.0, 28.0, 16.0, 4.0, 1...",92838.0
7,744622,"[50.0, 21.0, 8.0, 0.0, 0.0, 17.0, 15.0, 1.0, 4...",94371.0
8,815572,"[0.0, 2.0, 0.0, 2.0, 6.0, 11.0, 35.0, 17.0, 6....",96181.0
9,845758,"[24.0, 0.0, 0.0, 15.0, 4.0, 2.0, 15.0, 102.0, ...",97002.0


q => sample vector

k => how many neighbors to return

nprobes => how many partitions (in the coarse quantizer) to probe

refine_factor => controls "re-ranking". If k=10 and refine_factor=5 then retrieve 50 nearest neighbors by ANN and re-sort using actual distances then return top 10. This improves recall without sacrificing performance too much

**NOTE** the latencies above include file io as lance currently doesn't hold anything in memory. Along with index building speed, creating a purely in memory version of the dataset would make the biggest impact on performance.

### Features and vector can be retrieved together

Usually we have other feature or metadata columns that need to be stored and fetched together.
If you're managing data and the index separately, you to do a bunch of annoying plumbing to put stuff together. With Lance it's a single call

In [22]:
tbl = sift1m.to_table()
tbl = tbl.append_column("item_id", pa.array(range(len(tbl))))
tbl = tbl.append_column("revenue", pa.array((np.random.randn(len(tbl))+5)*1000))
tbl.to_pandas()

,id,vector,item_id,revenue
0,0,"[0.0, 16.0, 35.0, 5.0, 32.0, 31.0, 14.0, 10.0,...",0,4243.469146
1,1,"[1.8e-43, 14.0, 35.0, 19.0, 20.0, 3.0, 1.0, 13...",1,5544.403387
2,2,"[33.0, 1.8e-43, 0.0, 1.0, 5.0, 3.0, 44.0, 40.0...",2,5349.198998
3,3,"[23.0, 10.0, 1.8e-43, 12.0, 47.0, 14.0, 25.0, ...",3,4404.137670
4,4,"[27.0, 29.0, 21.0, 1.8e-43, 1.0, 1.0, 0.0, 0.0...",4,4975.389960
...,...,...,...,...
999995,999995,"[8.0, 9.0, 5.0, 0.0, 10.0, 39.0, 72.0, 68.0, 3...",999995,4474.915652
999996,999996,"[3.0, 28.0, 55.0, 29.0, 35.0, 12.0, 1.0, 2.0, ...",999996,3742.964504
999997,999997,"[0.0, 13.0, 41.0, 72.0, 40.0, 9.0, 0.0, 0.0, 0...",999997,5641.371131
999998,999998,"[41.0, 121.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24...",999998,5701.500194


In [23]:
sift1m = lance.write_dataset(tbl, uri, mode="overwrite")

In [24]:
sift1m.to_table(columns=["revenue"], nearest={"column": "vector", "q": samples[0], "k": 10}).to_pandas()

,revenue,vector,score
0,6180.891855,"[53.0, 51.0, 4.0, 0.0, 0.0, 0.0, 25.0, 52.0, 3...",0.0
1,5512.782565,"[30.0, 24.0, 10.0, 4.0, 4.0, 16.0, 15.0, 10.0,...",79038.0
2,4737.476359,"[44.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 25.0...",83997.0
3,6207.208024,"[41.0, 33.0, 13.0, 1.0, 0.0, 0.0, 0.0, 2.0, 14...",86389.0
4,5637.503015,"[0.0, 29.0, 10.0, 30.0, 12.0, 8.0, 4.0, 21.0, ...",86972.0
5,3658.692489,"[14.0, 59.0, 24.0, 0.0, 1.0, 0.0, 0.0, 12.0, 1...",87866.0
6,5273.637749,"[10.0, 6.0, 0.0, 3.0, 12.0, 28.0, 16.0, 4.0, 1...",92838.0
7,3833.229825,"[50.0, 21.0, 8.0, 0.0, 0.0, 17.0, 15.0, 1.0, 4...",94371.0
8,4826.840337,"[0.0, 2.0, 0.0, 2.0, 6.0, 11.0, 35.0, 17.0, 6....",96181.0
9,4060.225937,"[24.0, 0.0, 0.0, 15.0, 4.0, 2.0, 15.0, 102.0, ...",97002.0
